##  Using a list of all the valid DOIs, we will query the CrossRef API to acquire the lists of authors for each refernence and the publication date.

### then use these to generate an count of the number of authors 


#### This work could be added to the end of one of the other dataframes, but this appraoch should generate a cleaner set of nodes



In [1]:
# for data handling and to iterate though pairs of authors
import pandas as pd
import itertools

#to query Crossref
from habanero import Crossref, cn, counts, exceptions
import time


In [2]:
df1  =pd.read_csv('./C1in.csv',
                    index_col=['pub_date_CR_API'], parse_dates=True)
#pd.read_csv('./author_networks/all_dois.csv', header=None)
df1.shape

(2365, 17)

In [3]:
df1.head()

,Unnamed: 0,finaldoi_lower,ID,complete,FinalDOI,api_add,doi,Dim_times_cited,recent_citations,relative_citation_ratio,field_citation_ratio,license,CR_times_cited,authors_CR,year,month,auth_number
pub_date_CR_API,,,,,,,,,,,,,,,,,
2014-08-30 14:03:56+00:00,0,10.1186/s12881-014-0095-4,1125,"&amp; , fenwick al, goos jac, rankin j, lord h...",10.1186/s12881-014-0095-4,"{'doi': '10.1186/s12881-014-0095-4', 'times_ci...",10.1186/s12881-014-0095-4,7.0,4.0,0.24,0.78,This data has been sourced via the Dimensions ...,5.0,"[{'given': 'Aimee L', 'family': 'Fenwick', 'se...",2014.0,8.0,10.0
2016-06-01 01:53:39+00:00,1,10.1183/13993003.00321-2016,1996,", pattinson kt, turner mr. a wider pathologica...",10.1183/13993003.00321-2016,"{'doi': '10.1183/13993003.00321-2016', 'times_...",10.1183/13993003.00321-2016,4.0,3.0,0.57,0.99,This data has been sourced via the Dimensions ...,4.0,"[{'given': 'Kyle T.S.', 'family': 'Pattinson',...",2016.0,6.0,2.0
2013-05-15 04:36:29+00:00,2,10.1161/strokeaha.113.679936,506,"adib-samii p, rost n, traylor m, devan w, ...",10.1161/strokeaha.113.679936,"{'doi': '10.1161/strokeaha.113.679936', 'times...",10.1161/strokeaha.113.679936,37.0,4.0,1.37,8.71,This data has been sourced via the Dimensions ...,34.0,"[{'given': 'Poneh', 'family': 'Adib-Samii', 's...",2013.0,5.0,47.0
2014-09-12 00:32:37+00:00,3,10.1093/annonc/mdu449,1430,"and i. tomlinson*, findlay jm, middleton mr, t...",10.1093/annonc/mdu449,"{'doi': '10.1093/annonc/mdu449', 'times_cited'...",10.1093/annonc/mdu449,44.0,8.0,2.29,8.63,This data has been sourced via the Dimensions ...,33.0,"[{'given': 'J.M.', 'family': 'Findlay', 'seque...",2014.0,9.0,3.0
2013-11-22 05:07:47+00:00,4,10.1161/strokeaha.113.002707,848,"dichgans m, malik r, kã•_nig ir, rosand j, ...",10.1161/strokeaha.113.002707,"{'doi': '10.1161/strokeaha.113.002707', 'times...",10.1161/strokeaha.113.002707,230.0,62.0,8.70,54.15,This data has been sourced via the Dimensions ...,213.0,"[{'given': 'Martin', 'family': 'Dichgans', 'se...",2013.0,11.0,58.0


In [2]:
cr = Crossref()

In [3]:
# set a mailto address
Crossref(mailto = "your e-mail here")

< Crossref 
URL: https://api.crossref.org
KEY: None
MAILTO: your e-mail here
ADDITIONAL UA STRING: None
>

In [6]:
df1.iloc[1]

Unnamed: 0                                                                 1
finaldoi_lower                                   10.1183/13993003.00321-2016
ID                                                                      1996
complete                   , pattinson kt, turner mr. a wider pathologica...
FinalDOI                                         10.1183/13993003.00321-2016
api_add                    {'doi': '10.1183/13993003.00321-2016', 'times_...
doi                                              10.1183/13993003.00321-2016
Dim_times_cited                                                            4
recent_citations                                                           3
relative_citation_ratio                                                 0.57
field_citation_ratio                                                    0.99
license                    This data has been sourced via the Dimensions ...
CR_times_cited                                                             4

In [7]:
cr.works(ids=df1.iloc[1]['FinalDOI'])['message']

{'indexed': {'date-parts': [[2020, 4, 30]],
  'date-time': '2020-04-30T21:45:41Z',
  'timestamp': 1588283141427},
 'reference-count': 28,
 'publisher': 'European Respiratory Society (ERS)',
 'issue': '6',
 'funder': [{'DOI': '10.13039/501100000265',
   'name': 'Medical Research Council',
   'doi-asserted-by': 'publisher',
   'award': ['MR/K01014X/1']},
  {'DOI': '10.13039/501100000272',
   'name': 'National Institute for Health Research',
   'doi-asserted-by': 'publisher',
   'award': []}],
 'content-domain': {'domain': ['ersjournals.com'],
  'crossmark-restriction': True},
 'short-container-title': ['Eur Respir J'],
 'published-print': {'date-parts': [[2016, 6]]},
 'DOI': '10.1183/13993003.00321-2016',
 'type': 'journal-article',
 'created': {'date-parts': [[2016, 6, 1]],
  'date-time': '2016-06-01T01:53:39Z',
  'timestamp': 1464746019000},
 'page': '1632-1634',
 'update-policy': 'http://dx.doi.org/10.1183/ers-crossmark-policy',
 'source': 'Crossref',
 'is-referenced-by-count': 4,
 't

In [8]:
def DoiToRecord(df):
    try:
        search = cr.works(ids=df.FinalDOI)
        time.sleep(0.1)
        record = search ['message']
        return record
    except:
        pass
        

In [9]:
records = df1.apply(DoiToRecord, axis=1)

In [10]:
records

pub_date_CR_API
2014-08-30 14:03:56+00:00    {'indexed': {'date-parts': [[2020, 3, 29]], 'd...
2016-06-01 01:53:39+00:00    {'indexed': {'date-parts': [[2020, 4, 30]], 'd...
2013-05-15 04:36:29+00:00    {'indexed': {'date-parts': [[2020, 10, 24]], '...
2014-09-12 00:32:37+00:00    {'indexed': {'date-parts': [[2020, 7, 14]], 'd...
2013-11-22 05:07:47+00:00    {'indexed': {'date-parts': [[2021, 1, 27]], 'd...
                                                   ...                        
2016-02-16 23:57:27+00:00    {'indexed': {'date-parts': [[2021, 1, 15]], 'd...
2014-06-12 03:11:30+00:00    {'indexed': {'date-parts': [[2020, 12, 9]], 'd...
2014-07-04 04:37:33+00:00    {'indexed': {'date-parts': [[2020, 7, 20]], 'd...
2012-03-25 00:38:55+00:00    {'indexed': {'date-parts': [[2021, 1, 18]], 'd...
2012-04-26 22:45:19+00:00    {'indexed': {'date-parts': [[2020, 7, 7]], 'da...
Length: 2365, dtype: object

In [11]:
missing =df1[records.isna()]
missing#.to_clipboard()

,Unnamed: 0,finaldoi_lower,ID,complete,FinalDOI,api_add,doi,Dim_times_cited,recent_citations,relative_citation_ratio,field_citation_ratio,license,CR_times_cited,authors_CR,year,month,auth_number
pub_date_CR_API,,,,,,,,,,,,,,,,,
NaT,936,10.3978/j.issn.2225-319x.2014.05.14,981,"howard dp, sideso e, handa a, rothwell pm. inc...",10.3978/j.issn.2225-319X.2014.05.14,"{'doi': '10.3978/j.issn.2225-319x.2014.05.14',...",10.3978/j.issn.2225-319x.2014.05.14,35.0,16.0,1.30,NaN,This data has been sourced via the Dimensions ...,NaN,NaN,NaN,NaN,NaN
NaT,1289,10.3978/j.issn.2305-5839.2015.09.12,1723,"malavige gn, ogg gs.. role of skin homing t ce...",10.3978/j.issn.2305-5839.2015.09.12,"{'doi': '10.3978/j.issn.2305-5839.2015.09.12',...",10.3978/j.issn.2305-5839.2015.09.12,0.0,0.0,0.00,NaN,This data has been sourced via the Dimensions ...,NaN,NaN,NaN,NaN,NaN
NaT,1826,10.1002/cyto.b.21165,1140,"sadler r, bateman ea, heath v, patel sy, schwi...",10.1002/cyto.b.21165,"{'doi': '10.1002/cyto.b.21165', 'times_cited':...",10.1002/cyto.b.21165,11.0,3.0,0.42,1.34,This data has been sourced via the Dimensions ...,NaN,NaN,NaN,NaN,NaN
NaT,1828,10.5083/ejcm.20424884.147,2198,"sahan km, channon km, choudhury rp, kharbanda ...",10.5083/ejcm.20424884.147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
records_all =records.dropna()#.append(records_retry)
records_df = pd.DataFrame(pd.io.json.json_normalize(records_all.dropna()))

In [13]:
records_df.to_json('./D1out_CrossRef_records.json')
records_df.shape

(2361, 66)